<a href="https://colab.research.google.com/github/EricDarve/cme216-spring-2020/blob/master/Code/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly==4.4.1

In [0]:
import math
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

In [0]:
# +1 if x > y and -1 otherwise
def labels_line(p):
  if p[0] > p[1]:
    return 1
  else:
    return -1

# +1 if y > sin(5x)/2 and -1 otherwise
def labels_sine(p):
  if p[1] > math.sin(5*p[0]) / 2:
    return 1
  else:
    return -1   

In [0]:
def plot_data_decision_line(X, y, clf, fig, title):
  fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1],
                mode='markers', name='Data', 
                marker = dict(color=y, colorscale="RdBu", size=10)))

  x = np.linspace(-1, 1, 2)

  a = -clf.coef_[0,0] / clf.coef_[0,1]
  b0 = -clf.intercept_ / clf.coef_[0,1]
  b1 = -(1 + clf.intercept_) / clf.coef_[0,1]
  b2 = -(-1 + clf.intercept_) / clf.coef_[0,1]

  fig.add_trace(go.Scatter(x=x, y=a*x + b0,
                      mode='lines', name='Decision line'))
  fig.add_trace(go.Scatter(x=x, y=a*x + b1,
                      mode='lines', name='Support line',
                      line=dict(dash="dash")))
  fig.add_trace(go.Scatter(x=x, y=a*x + b2,
                      mode='lines', name='Support line',
                      line=dict(dash="dash")))
  fig.update_layout(title=title, width=600,
                    yaxis = dict(scaleanchor = "x", scaleratio = 1))  

def plot_data_decision_function(X, y, clf, fig, title):

    # plot the decision function
    xm = np.linspace(-1, 1, 20)
    ym = np.linspace(-1, 1, 20)
    xx, yy = np.meshgrid(xm,ym)

    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    # Computes the decision function that separates the 1 labels 
    # from the -1 labels
    # ravel() returns a contiguous flattened array
    # c_ concatenates arrays along the 2nd dimension
    Z = Z.reshape(xx.shape)

    fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1],
                  mode='markers', showlegend=False, 
                  marker = dict(color=y, colorscale="RdBu", size=10)))
    fig.add_trace(go.Contour(x=xm,y=ym,z=Z,colorscale='RdBu'))
    fig.update_layout(title=title, width=600, 
                      yaxis = dict(scaleanchor = "x", scaleratio = 1))

In [5]:
# we create 8 random uniform points in the unit square

np.random.seed(2)
X = 2*np.random.rand(8, 2) - 1

y = np.zeros(X.shape[0])

# initialize labels
for i in range(0,X.shape[0]):
  y[i] = labels_line(X[i,:])

print('Shape of X: ', X.shape)
print(X)
print('Shape of y: ', y.shape)
print(y)

clf = svm.SVC(kernel='linear', C=1e2)
clf.fit(X, y)

fig = go.Figure()
plot_data_decision_line(X,y,clf,fig,"SVM")
fig.show()

Shape of X:  (8, 2)
[[-0.1280102  -0.94814754]
 [ 0.09932496 -0.12935521]
 [-0.1592644  -0.33933036]
 [-0.59070273  0.23854193]
 [-0.40069065 -0.46634545]
 [ 0.24226767  0.05828419]
 [-0.73084011  0.02715624]
 [-0.63112027  0.5706703 ]]
Shape of y:  (8,)
[ 1.  1.  1. -1.  1.  1. -1. -1.]


In [6]:
# 20 uniform points in the unit square
# the decision function is plotted

np.random.seed(2020)
X = 2*np.random.rand(32, 2) - 1

y = np.zeros(X.shape[0])

# initialize labels
for i in range(0,X.shape[0]):
  y[i] = labels_line(X[i,:])

# fit the model
clf = svm.SVC(kernel="linear")
clf.fit(X, y)

# plot
fig = go.Figure()
plot_data_decision_function(X, y, clf, fig, "Decision function")
fig.show()

In [7]:
# 20 uniform points in the unit square
# noise is added to the coordinates of the points

np.random.seed(2020)
X = 2*np.random.rand(32, 2) - 1

y = np.zeros(X.shape[0])

# initialize labels
for i in range(0,X.shape[0]):
  y[i] = labels_line(X[i,:])

# randomly perturb the points X
for i in range(0,X.shape[0]):
  X[i,:] = X[i,:] + ( 2*np.random.rand(1, 2) - 1 ) / 2

# fit the model
clf = svm.SVC(kernel="linear")
clf.fit(X, y)

# plot
fig = go.Figure()
plot_data_decision_function(X, y, clf, fig, "Decision function")
fig.show()

In [8]:
# plot points and decision line
fig = go.Figure()
plot_data_decision_line(X,y,clf,fig,"With noise")
fig.show()

In [9]:
# 2 outlier points
np.random.seed(2020)

# two clusters of points
X = np.r_[np.random.rand(10, 2)/2 + [-0.6, 0.1], 
          np.random.rand(10, 2)/2 + [0.1, -0.6]]

y = np.zeros(X.shape[0])

# initialize labels
for i in range(0,X.shape[0]):
  y[i] = labels_line(X[i,:])

# creating two outliers
X[0,:] = np.array([0.5, 0])
y[0] = -1
X[1,:] = np.array([-0.5, 0])
y[1] = 1

# plot the points
fig = go.Figure()
fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1],
              mode='markers', name='Data', 
              marker = dict(color=y, colorscale="RdBu", size=10)))
fig.update_layout(width=600,
                  yaxis = dict(scaleanchor = "x", scaleratio = 1))  
fig.show()

In [10]:
# 2 outlier points
# Large value of C = overfit

np.random.seed(2020)

# two clusters of points
X = np.r_[np.random.rand(10, 2)/2 + [-0.6, 0.1], 
          np.random.rand(10, 2)/2 + [0.1, -0.6]]

y = np.zeros(X.shape[0])

# initialize labels
for i in range(0,X.shape[0]):
  y[i] = labels_line(X[i,:])

# creating two outliers
X[0,:] = np.array([0.5, 0])
y[0] = -1
X[1,:] = np.array([-0.5, 0])
y[1] = 1

# fit the model
clf = svm.SVC(kernel="linear", C = 1e4)
clf.fit(X, y)

fig = go.Figure()
plot_data_decision_line(X,y,clf,fig,"C=10^4")
fig.show()

In [11]:
# outliers

# small C; under-fitting case; prediction is not accurate

# fit the model
clf = svm.SVC(kernel="linear", C = 0.2)
clf.fit(X, y)

fig = go.Figure()
plot_data_decision_line(X,y,clf,fig,"C=0.2")
fig.show()

In [12]:
# outliers

# fit the model
clf = svm.SVC(kernel="linear", C = 0.3)
clf.fit(X, y)

fig = go.Figure()
plot_data_decision_line(X,y,clf,fig,"C=0.3")
x_l = np.linspace(-1, 1, 2)
fig.add_trace(go.Scatter(x=x_l, y=x_l, mode='lines', name='y=x'))
fig.show()

In [13]:
# Data with noise
# noise is added to the coordinates of the points

np.random.seed(2020)
X = 2*np.random.rand(32, 2) - 1

y = np.zeros(X.shape[0])

# initialize labels
for i in range(0,X.shape[0]):
  y[i] = labels_line(X[i,:])

# randomly perturb the points X
for i in range(0,X.shape[0]):
  X[i,:] = X[i,:] + ( 2*np.random.rand(1, 2) - 1 ) / 2

# fit the model
clf = svm.SVC(kernel="linear",C=1)
clf.fit(X, y)

# plot
fig = go.Figure()
plot_data_decision_function(X, y, clf, fig, "Decision function")
fig.show()

In [0]:
np.random.seed(2020)

X = np.random.randn(256, 2)
y = np.zeros(X.shape[0])

# initialize labels
for i in range(0,X.shape[0]):
  y[i] = labels_line(X[i,:])

# randomly perturb the points X
for i in range(0,X.shape[0]):
  X[i,:] = X[i,:] + np.random.randn(2)/4

n_opt_C = 128

C_range = np.logspace(-3,1,n_opt_C)
score = np.zeros(n_opt_C)
a = np.zeros(n_opt_C)
b = np.zeros(n_opt_C)

k = 0
for C in C_range:
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4)
  clf = svm.SVC(kernel='linear', C=C).fit(X_train, y_train)
  score[k] = clf.score(X_valid, y_valid)
  a[k] = -clf.coef_[0,0] / clf.coef_[0,1]
  b[k] = -clf.intercept_ / clf.coef_[0,1]

  k = k+1

In [15]:
# plot
from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=(
                        "Data", "Score", "Slope a", "Intercept b"
                        ))
                    
fig.add_trace(go.Scatter(x=X[:, 0], y=X[:, 1],
              mode='markers', name='Data', 
              marker = dict(color=y, colorscale="RdBu", size=10)),
              row=1, col=1)
x_l = [-2, 2]
fig.add_trace(go.Scatter(x=x_l, y=x_l,
              mode='lines', name='y=x'),
              row=1, col=1)

fig.add_trace(go.Scatter(x=C_range, y=score,
              mode='lines', name='Score'),
              row=1, col=2)

fig.add_trace(go.Scatter(x=C_range, y=a,
              mode='lines', name='a'),
              row=2, col=1)
fig.add_trace(go.Scatter(x=C_range, y=np.ones(C_range.shape),
              mode='lines', name='a=1'),
              row=2, col=1)

fig.add_trace(go.Scatter(x=C_range, y=b,
              mode='lines', name='b'),
              row=2, col=2)
fig.add_trace(go.Scatter(x=C_range, y=np.zeros(C_range.shape),
              mode='lines', name='b=0'),
              row=2, col=2)

fig.update_layout(height=700,width=1000)  
fig.update_yaxes(scaleanchor = "x", scaleratio = 1, row=1, col=1)
fig.update_xaxes(type="log", row=1, col=2)
fig.update_xaxes(type="log", row=2, col=1)
fig.update_xaxes(type="log", row=2, col=2)
fig.update_yaxes(range=[-0.5, 0.5], row=2, col=2)

fig.show()

In [16]:
np.random.seed(2020)

# decision curve is the sine function

X = 2*np.random.rand(128, 2) - 1

y = np.zeros(X.shape[0])

# initialize labels
for i in range(0,X.shape[0]):
  y[i] = labels_sine(X[i,:])

# fit the model
# we use a linear SVM which won't work in this case
clf = svm.SVC(kernel="linear")
clf.fit(X, y)

fig = go.Figure()
plot_data_decision_function(X, y, clf, fig, "Decision function")
# draw the sine function
x_b = np.linspace(-1, 1, 32)
fig.add_trace(go.Scatter(x=x_b, y=np.sin(5*x_b) / 2,
                    mode='lines', showlegend=False))
fig.show()

In [17]:
# SVM using RBF kernel

# fit the model
clf = svm.SVC(kernel="rbf", gamma = 10)
clf.fit(X, y)

fig = go.Figure()
plot_data_decision_function(X, y, clf, fig, "Decision function")
# draw the sine function
x_b = np.linspace(-1, 1, 32)
fig.add_trace(go.Scatter(x=x_b, y=np.sin(5*x_b) / 2,
                    mode='lines', showlegend=False))
fig.show()